In [ ]:
import numpy as np
from importnb import Notebook

with Notebook(): 
    import reproduccion_ulutas as rp

In [8]:
POPULATION_SIZE = 100   # indicate the initial size of antibodies population
# N is a random % of antibodies with highest affinities selected to be cloned
R = 0.1 # % of best cloned antibodies to the pool of antibodies
B = 0.1 # worst % of the whole population (RECEPTOR EDITING)
dataset = 'data/instances/testset_a/5x7_Waghodekar_Sahu(1984)[Problem-2].txt'
MAX_ITERATIONS = 1000
MAX_ITERATIONS_WITHOUT_IMPROVEMENT = 1000
log = False # Boolean: print logs at the execution of the algorithm.

## 1. ~~Select N% of antibodies with highest affinities.~~
        SELECTION ROULETTE WHEEL: antibodies that have higher efficacy values have a higher selection probability.
        N is a random number at each iteration
        
        Esos anticuerpos no se duplican, se clonan, se mutan y, si mejora su eficacia, son actualizados en el pool de anticuerpos

## 2. Incluir semilla en la **mutación** para que los resultados sean reproducibles
        

## 3. ~~Add new random antibodies into the population~~ 
~~La misma cantidad de anticuerpos que hemos borrado en el apartado anterior~~

## 4. Termination criteria
        Algorithm is terminated if no improvement is obtained in the efficacy value after 1000 iterations

## 5. ~~Penalties: calculation of affinity.~~
        each cell must contain at least one part and one machine. (cezali*(efficacy/2)) + (cresing*(efficacy/2))
        
        During calculation of the efficacy value, a penalty is defined because Cell#2 does not include any part. 
        Likewise, if a cell does not include any machine, a penalty value is defined and antibody’s objective function value gets a lower efficacy value. 
        The penalty value reduces the chance of the antibody to be selected in the next iteration
        

## 6. ~~Mutation: if new efficacy value is higher, antibody is updated.~~
        Parece que la AFFINITY solo se utiliza en el primer paso del algoritmo, para hacer la selección del N% para la mutación

In [9]:
#reads data and translates it into part-machine matrix
matrix, m, p, columns, rows = rp.part_machine_incidence_matrix(dataset)
if log == True: print("\nPart-machine matrix\n",matrix)
#GENERATION OF INITIAL POPULATION
antibodies = rp.generation_initial_population(p = p, m= m, population_size = POPULATION_SIZE, seed = 2021)
if log == True: print("\nAnticuerpos iniciales\n",antibodies)

iterations, best_solution, improvement_counter = 0, 0, 0

while iterations <= MAX_ITERATIONS:
    if log == True: print("\n Iteration number",iterations+1)
    #identifies cells in the antibodies
    total_cells = rp.cell_identification(antibodies = antibodies)
    if log == True: print("\n Cells\n", total_cells)

    #translates numbers into parts and machines
    total_machines, total_parts = rp.decode_cells(total_cells = total_cells) 

    #calculates part-machine matrix for each antibody
    antibody_matrices = rp.create_machine_part_matrix(matrix = matrix, 
                                                    antibodies = antibodies, 
                                                    total_machines = total_machines, 
                                                    total_parts = total_parts)
    #evaluates efficacie of each antibody
    efficacies, affinities, voids, exceptions = rp.evaluate_antibodies(antibody_matrices = antibody_matrices, total_cells = total_cells)
    if log == True: print("\n Efficacies",efficacies)
    if log == True: print("\n Affinities",affinities)

    # SELECTS N% OF ANTIBODIES WITH HIGHES ANTIBODIES (SELECTION ROULETTE WHEEL)
    N = np.random.rand(1) # N is a random number each iteration
    if log: print("N% =",N[0])
    cloned_antibodies, positions_antibodies_selected = rp.antibodies_selection(antibodies=antibodies, N=N[0], affinities = affinities)
    if log == True: print("\n Cloned antibodies\n",cloned_antibodies)
    if log == True: print("\n Cloned antibodies positions\n",positions_antibodies_selected)
    
    # MUTATION
    rp.mutate_cloned_antibodies(cloned_antibodies = cloned_antibodies)
    if log == True: print("\n Cloned and mutated antibodies\n",cloned_antibodies)

    #CLONES: identifies cells in the antibody
    cloned_total_cells = rp.cell_identification(cloned_antibodies)
    #CLONES: translates numbers into parts and machines
    cloned_total_machines, cloned_total_parts = rp.decode_cells(cloned_total_cells)
    #CLONES: calculates part-machine matrix for each antibody
    cloned_antibody_matrices = rp.create_machine_part_matrix(matrix = matrix,
                                                                antibodies = cloned_antibodies, 
                                                                total_machines = cloned_total_machines, 
                                                                total_parts = cloned_total_parts)                                                          
    #CLONES evaluates efficacie of each antibody
    cloned_efficacies, cloned_affinities, cloned_voids, cloned_exceptions = rp.evaluate_antibodies(cloned_antibody_matrices,
                                                                                                    cloned_total_cells)
    if log == True: print("\n Cloned Efficacies",cloned_efficacies)
    if log == True: print("\n Cloned Affinities",cloned_affinities)
    # Add R% of best cloned antibodies to the pool of antibodies
    antibodies, efficacies, affinities = rp.select_best_cloned_antibodies(antibodies = antibodies,
                                        cloned_antibodies = cloned_antibodies,
                                        efficacies = efficacies, 
                                        cloned_efficacies = cloned_efficacies,
                                        affinities = affinities, 
                                        cloned_affinities = cloned_affinities,
                                        R = R,
                                        positions_antibodies_selected = positions_antibodies_selected)
    if log == True: print("\n New antibodies pool \n",antibodies)                                                                        
    # RECEPTOR EDITING: Remove worst members of the antibodies pool
    antibodies, efficacies, affinities, amount_antibodies_erased = rp.receptor_editing(antibodies_pool = antibodies, 
                                                                                efficacies = efficacies, 
                                                                                affinities=affinities, 
                                                                                B = B)
    if log == True: print("\n Antibodies pool after removal \n", antibodies)
    #GENERATION OF NEW RANDOM ANTIBODIES
    number_new_random_antibodies = amount_antibodies_erased
    new_random_antibodies = rp.generation_initial_population(p = p, 
                                                        m = m, 
                                                        population_size = number_new_random_antibodies, 
                                                        seed = 1995)
    if log == True: print("\n New random antibodies \n",new_random_antibodies)                                                        
    antibodies = np.concatenate((antibodies, new_random_antibodies), axis = 0)
    if log == True: print("\n Antibodies pool after adding new random antibodies \n",antibodies)
    
    #TERMINATION CRITERIA AND SELECTION OF BEST SOLUTION
    iteration_best_solution = np.amax(efficacies) #max value efficacy for this iteration
    index_best_solution = np.argmax(efficacies)
    best_antibody = antibodies[index_best_solution]
    if log: print("\n Best solution at the moment is {} (antibody {}, index {})".format(iteration_best_solution,best_antibody,index_best_solution))
    if iteration_best_solution <= best_solution:
        improvement_counter = improvement_counter + 1
        if improvement_counter >= MAX_ITERATIONS_WITHOUT_IMPROVEMENT:
            print("Maximun number of iterations without improvement reached: {}".format(MAX_ITERATIONS_WITHOUT_IMPROVEMENT))
            print("Best solution obtained is {}".format(best_solution))
            break
    else:
        best_solution = iteration_best_solution
    #update iteration counter
    iterations = iterations + 1

decoded antibodies [['P4'], ['P3', 'M4', 'P2', 'M1'], ['M5', 'M2'], ['P7', 'P1', 'M3', 'P6', 'P5']]
decoded antibodies [['P3', 'P1', 'P5'], ['M3', 'P7', 'M5', 'P6', 'M1', 'P4', 'M2', 'P2', 'M4']]
decoded antibodies [['P4', 'M4', 'P5', 'P6', 'P1', 'M3', 'M2', 'M1', 'P7', 'P2'], ['P3', 'M5']]
decoded antibodies [['P5', 'P3'], ['M4', 'P2', 'P4', 'M2', 'M5'], ['M1'], ['M3'], ['P7', 'P1', 'P6']]
decoded antibodies [['M4'], ['P6'], ['P3', 'M5', 'P7', 'M3'], ['P1', 'M2', 'P5', 'M1', 'P4', 'P2']]
decoded antibodies [['P5', 'P2', 'P6', 'P1'], ['P7', 'M1', 'M2', 'M3', 'P4', 'M5', 'M4'], ['P3']]
decoded antibodies [['P6', 'M4'], ['P2', 'P4', 'P5', 'M1'], ['M5', 'M2', 'P3', 'P7'], ['M3'], ['P1']]
decoded antibodies [['P2'], ['P1'], ['P3', 'M4'], ['M3', 'P7', 'M5'], ['M1', 'P6', 'M2', 'P5', 'P4']]
decoded antibodies [['P6', 'P4', 'P1'], ['M3', 'M2', 'M1', 'P7', 'P5'], ['P2', 'M5', 'M4', 'P3']]
decoded antibodies [['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'M2', 'M5'], ['M1', 'M4', 'M3'], ['P7']]
decoded 

In [14]:
best_solution

0.7631578947368421

In [11]:
best_antibody

array([ 1,  6,  0,  8,  7,  0,  4,  3, 12,  2,  9, 11, 10,  5,  0,  0])

In [13]:
cell_identified_best_antibody = rp.cell_identification([best_antibody])
total_machines, total_parts = rp.decode_cells(cell_identified_best_antibody)

antibody_matrices = rp.create_machine_part_matrix(matrix=matrix,
                                                antibodies=[best_antibody], 
                                                total_machines=total_machines, 
                                                total_parts=total_parts)
antibody_matrices                                                

decoded antibodies [['P1', 'P6'], ['M1', 'P7'], ['P4', 'P3', 'M5', 'P2', 'M2', 'M4', 'M3', 'P5']]


[    M1  M5  M2  M4  M3
 P1   1   0   0   1   0
 P6   1   1   0   0   1
 P7   1   0   0   0   0
 P4   0   1   1   1   1
 P3   0   0   1   1   1
 P2   0   1   1   1   0
 P5   1   1   1   0   1]

In [15]:
efficacies, affinities, voids, exceptions = rp.evaluate_antibodies(antibody_matrices = antibody_matrices, 
                                                                    total_cells=cell_identified_best_antibody)
efficacies

[0.7631578947368421]